# Image Classification

In [162]:
import os
from pathlib import Path
import sys

from typing import List,Dict,Optional,Union,Tuple,Any,Iterable

import torch
from torch.utils.data import Dataset,DataLoader
import wandb

import numpy as np
from scipy import stats
from tqdm.notebook import tqdm,trange

import time

import pickle
import matplotlib.pyplot as plt

In [177]:
def timeit(func):
    
    def wrapped_func(*args,**kwargs):
        
        start = time.perf_counter()
        res = func(*args,**kwargs)
        end = time.perf_counter()
        
        print(f"{func.__name__} took {(end-start):>1f}s to run")
    
    return wrapped_func

## Loading the Data

### CIFAR 10

    Image classification dataset with 10 classes
    
    Classes : 10
    Train   : 50,000
    Test    : 10,000

In [17]:
class Cifar10(Dataset):
    
    def __init__(self,data_dir:str,train:bool=True,transform=None,label_transform=None):
        
        self.data_dir = data_dir
        
        self.Xs = np.array([])
        self.ys = np.array([])
        self.names = {}
        
        self.transform = transform
        self.label_transform = label_transform

        self.label_names = {  
            0: "airplane",
            1: "automobile",
            2: "bird",
            3: "cat",
            4: "deer",
            5: "dog",
            6: "frog",
            7: "horse",
            8: "ship" ,
            9: "truck",
        }
        
        def filename_to_name_idx(name):
            name = str(name).split('_')
            return name[-1],name[0]
        
        if train:
            files = filter(lambda file: 'data' in file, os.listdir(self.data_dir))
        else:
            files = filter(lambda file: 'test' in file, os.listdir(self.data_dir))

        count = 0
        for file in files:
            data,labels,names = self.unpickle(os.path.join(self.data_dir,file))
            if count == 0:
                self.Xs = np.array(data)
                self.ys = np.array(labels)
                count += 1
            else:
                self.Xs = np.vstack((self.Xs,data))
                self.ys = np.vstack((self.ys,labels))

            names = {idx:name for idx,name in map(filename_to_name_idx,names)}

            self.names.update(names)
        
        self.ys = self.ys.reshape(-1)
        assert self.Xs.shape[0]==self.ys.shape[0],f"Data and labels are not in same shape {self.Xs.shape,self.ys.shape}"
        
    def __len__(self) -> int:
        return self.Xs.shape[0]
    
    def __getitem__(self,idx) -> Tuple[np.ndarray,Union[int,np.ndarray]]:
        image = self.Xs[idx]
        label = self.ys[idx]
        if self.transform:
            image = self.transform(image)
        if self.label_transform:
            label = self.label_transform(label)
        
        return image,label
    
    def show_example(self,idx:int) -> None:
        img,label = self.__getitem__(idx)
        plt.imshow(img.transpose(1,2,0))
        plt.title(self.label_names[label])
        plt.axis('off')
        plt.show()
        
    def show_random_example(self) -> None:
        idx = np.random.randint(0,self.__len__())
        self.show_example(idx)
        
    def get_random_grid(self,grid_size:int = 5,viz:bool=False) -> np.ndarray:
        
        idx = lambda : np.random.randint(0,len(self))
        img = np.concatenate([ np.concatenate( [ self[idx()][0].transpose(1,2,0) for _ in range(grid_size)] ,axis=1) for _ in range(grid_size) ],axis=0)
        if viz:
            fig = plt.figure(figsize=(7,7))
            plt.imshow(img)
            plt.axis('off')
            plt.show()
        else:
            return img
            
    @staticmethod
    def unpickle(filename:str) -> Any:
        with open(filename,'rb') as file:
            batch = pickle.load(file,encoding='bytes')
        
        try:
            labels = batch[b'labels']
            data = batch[b'data'].reshape(-1,3,32,32)
            names = batch[b'filenames']
        except:
            print(type(batch))
        
        return data,labels,names



In [18]:
data_dir = Path('../data/cifar-10-batches-py/')
files = os.listdir(data_dir)


#### Train/Val/Test split

In [19]:
ds_train = Cifar10(data_dir)
ds_test = Cifar10(data_dir,train=False)

split = 0.9
train_set,val_set = torch.utils.data.random_split(
    ds_train,
    ( int(len(ds_train)*(split)) , int(len(ds_train)*(1-split)) + 1 )
)

#### Using Pytorch Dataset and Dataloader for ease access

In [30]:
Bs = 32

train_loader = DataLoader(
    dataset=train_set,
    batch_size=Bs,
    shuffle=True
)

train_loader = DataLoader(
    dataset=val_set,
    batch_size=Bs,
    shuffle=True
)

test_loader = DataLoader(
    dataset=ds_test,
    batch_size=Bs,
    shuffle=True
)

## Classification

### k-NN classification
    
- Categorize all the training examples
- While Predicting -> find $k$ nearest neighours and return common class of the neighbours
    
#### Metrics

##### L1 distance
    Manhattan Distance
    Used where individual elements makes meaning like (emp salary, age, etc)
    
$$ L_{1} = \sum_{i=i}^{n}|{x_{i}}| $$ 

##### L2 distance (np default) 
    Forebinus norm, Euclid Distance
    
$$ L_{2} = \sqrt{\sum_{i=i}^{n}{|{x_{i}}|}^2} = \|x\|$$

In [180]:
class KNN:
    
    def __init__(self,dataset):
        
        self.dataset = dataset
        self.Xs,self.ys = self.dataset[:]
    
    def forward(self):
        # Just memorizing all the data
        pass
        
    def predict(self,img,metric:bool='l2',k:int=1,viz:bool=False):
        
        # ineffective
        if metric == 'l1':
            distances = np.sum(np.abs((X - img).reshape(X.shape[0],-1)),axis=-1)
        elif metric=='l2':
            distances = np.linalg.norm((X - img).reshape(X.shape[0],-1)**2,axis=-1)
            
        if k > 1:
            nearest_k = (-distances).argsort()[:n]
            nearest = nearest_k[0]
        else:
            nearest = distances.argmin(axis=0)
            pred = self.ys[nearest]
        
        if viz:
            fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(7,7))
            ax[0].imshow(self.Xs[nearest].transpose(1,2,0))
            ax[0].set_title(self.ys[nearest])
            ax[1].imshow(img.transpose(1,2,0))
            ax[1].set_title(f"Pred as :{self.ys[nearest]}")
            plt.show()
            
        return self.ys[nearest]
    
    @timeit
    def test(self,size:int=1000):
        
        crct = 0
        
        # very ineffective cause for every n test example it has to find diff for M train examples
        for idx in np.random.randint(0,len(knn.dataset),(size)):
            img,label = knn.dataset[idx]
            aug_x = img + np.random.randint(0,150,(3,32,32))
            aug_x = 255 * (aug_x - aug_x.min())/(aug_x.max()-aug_x.min())
            
            res = self.predict(np.int32(aug_x),metric='l1')
            crct += int(res == label)
        
        print(f"Acc : {crct/size *100}% for size {size}")
        
knn = KNN(train_set)

#### Test Runs

In [181]:
knn.test(10)

Acc : 70.0% for size 10
test took 4.595266s to run


### Linear

#### Parametric model
- Find a meaningful function h (linear)
- Find the best fiting $\theta$ for the given train set $X$
$$ \hat{y} = h(X,\theta) $$
- Define the Loss func $J$ such that the goal is
$$ \underset{\theta}{argmin}\ {J(X,\theta)} $$
- Traning with convex optimization $\eta$ is learning_rate
$$ \theta\ \leftarrow\ \theta - \eta.{J'(X,\theta))}$$

#### Linear Layer

- Notation
    
    - No. Example : $m,i$
    - No. feature : $n,j$
    - Input       : $X \in {\mathbb{R^{n*m}}}$
    - Label       : $y \in {\mathbb{R^{k}}}$ $k$ classes
    - Prediction  : $\hat{y}$
    - Weights.    : $\theta \in {\mathbb{R^{n*k}}}$
    - Biases.     : $b \in {\mathbb{R^{n*1}}}$
    - Loss fn     : $J$
    - L.Rate      : $\eta$
- hypothesis is kinda a straight line eqation
$$ h(X,\theta,b) = \theta^TX\ + b = \hat{y}$$
- Loss function is MSE  where $\lambda$ is regularization
$$ J(y,\hat{y}) = \frac{1}{2m}\sum_{i=1}^{m}\sum_{j=1}^{n}{(y_{j}^{(i)}-\hat{y}_{j}^{(i)})}^2 + \frac{\lambda}{2m}\sum_{j=1}^{n}(\theta_j)^2$$
- Optimization : SGD
$$ \theta_j \leftarrow \theta_j - \eta.{J'(y,\hat{y})_\theta}$$
$$ b_j \leftarrow b_j - \eta.{J'(y,\hat{y})_b}$$